## RunnableParallel

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [2]:
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "gemma3" #"llama3.2:3b"

max_tokens = 100
llm = ChatOllama(model=model_name, num_predict=max_tokens, temperature=0)

In [12]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [4]:
string_parser = StrOutputParser()

In [5]:
chain_books = chat_template_books | llm | string_parser

chain_projects = chat_template_projects | llm | string_parser

In [6]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [7]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1.  *Python Crash Course* by Eric Matthes\n2.  *Automate the Boring Stuff with Python* by Al Sweigart\n3.  *Effective Python: 90 Specific Ways to Write Better Python* by Brett Slatkin',
 'projects': '1.  Web Scraper and Data Analysis Tool\n2.  Real-Time Chat Application (using websockets)\n3.  Command-Line RPG Game\n'}

In [8]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOllama |                      | ChatOllama |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [9]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: total: 46.9 ms
Wall time: 6.61 s


'1.  *Python Crash Course* by Eric Matthes\n2.  *Automate the Boring Stuff with Python* by Al Sweigart\n3.  *Effective Python: 90 Specific Ways to Write Better Python* by Brett Slatkin'

In [10]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 31.2 ms
Wall time: 4.6 s


'1.  Web Scraper and Data Analysis Tool\n2.  Real-Time Chat Application (using websockets)\n3.  Command-Line RPG Game\n'

In [11]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 11 s


{'books': '1.  *Python Crash Course* by Eric Matthes\n2.  *Automate the Boring Stuff with Python* by Al Sweigart\n3.  *Effective Python: 90 Specific Ways to Write Better Python* by Brett Slatkin',
 'projects': '1.  Web Scraper and Data Analysis Tool\n2.  Real-Time Chat Application (using websockets)\n3.  Command-Line RPG Game\n'}

## Piping a RunnableParallel with Other Runnables

In [14]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | llm 
              | string_parser
             )

In [18]:
print(chain_time1.invoke({'programming language':'Python'}))

Okay, let's break down the estimated time investment for you, an intermediate programmer, to complete the suggested literature and projects. This is going to be a rough estimate, as individual learning speeds and project complexity can vary significantly.

**1. Literature Completion:**

* **Python Crash Course (Eric Matthes):**  This is a fantastic starting point.  Given your intermediate level, you should be able to work through it relatively quickly.
    * **Estimated Time:** 1


In [15]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | llm 
              | string_parser
             )

In [16]:
print(chain_time2.invoke({'programming language':'Python'}))

Okay, let's break down the estimated time investment for you, an intermediate programmer, to complete the suggested literature and projects. This is going to be a rough estimate, as individual learning speeds and project complexity can vary significantly.

**1. Literature Completion:**

* **Python Crash Course (Eric Matthes):**  This is a fantastic starting point.  Given your intermediate level, you should be able to work through it relatively quickly.
    * **Estimated Time:** 1


In [17]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOllama |                      | ChatOllama |     
    +------------+                      +------------+     
           *                                   *           
           *                            